
# PROJET ML Garnier Grenier

Importation des données ELection 2022 2znd tour

In [1]:
# -*- coding: utf-8 -*-
# ESLint-disable
import pandas as pd
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import classification_report
from sklearn.feature_selection import VarianceThreshold
from sklearn import metrics
from sklearn.model_selection import cross_val_score

In [2]:
# Create a DataFrame from the CSV data
data = pd.read_excel('https://www.data.gouv.fr/fr/datasets/r/06d9816c-1b87-498d-985e-f312acee4f51')

df = data.copy()
df = df.drop(df.columns[[26, 27, 29, 31, 32]], axis=1)
df = df.drop(['Code du département', 'Code de la commune', 'N°Panneau', 'Sexe', 
              'Prénom', '% Abs/Ins', '% Vot/Ins', '% Blancs/Ins', '% Blancs/Vot', '% Nuls/Ins', 
              '% Nuls/Vot', '% Exp/Ins', '% Exp/Vot', '% Voix/Ins', '% Voix/Exp'], axis=1,)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
df.head()

,Libellé du département,Libellé de la commune,Etat saisie,Inscrits,Abstentions,Votants,Blancs,Nuls,Exprimés,Nom,Voix,Unnamed: 28,Unnamed: 30
0,Ain,L'Abergement-Clémenciat,Complet,643,146,497,42,5,450,MACRON,237,LE PEN,213
1,Ain,L'Abergement-de-Varey,Complet,213,45,168,23,3,142,MACRON,94,LE PEN,48
2,Ain,Ambérieu-en-Bugey,Complet,8763,2448,6315,461,165,5689,MACRON,3080,LE PEN,2609
3,Ain,Ambérieux-en-Dombes,Complet,1282,266,1016,47,15,954,MACRON,451,LE PEN,503
4,Ain,Ambléon,Complet,103,19,84,12,0,72,MACRON,45,LE PEN,27


In [3]:
df.var(axis = 0)

/tmp/cache-anagarnier/ipykernel_2708846/1637361517.py:1: FutureWarning: The default value of numeric_only in DataFrame.var is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.var(axis = 0)


Inscrits       8.762086e+07
Abstentions    7.612050e+06
Votants        4.480993e+07
Blancs         1.300221e+05
Nuls           1.326667e+04
Exprimés       3.883480e+07
Voix           2.476095e+07
Unnamed: 30    2.162080e+06
dtype: float64

In [4]:
X = df[['Inscrits', 'Abstentions', 'Blancs', 'Nuls']]  # variables indépendantes
y = df['Voix']  # variable dépendante

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
r2_score = metrics.r2_score(y_test, y_pred)
scores = cross_val_score(model, X, y, cv=5)

nouvelles_donnees = pd.DataFrame({'Inscrits': [1000], 'Abstentions': [200], 'Blancs': [30], 'Nuls': [10]})
prediction = model.predict(nouvelles_donnees)

# Imprimer les scores de chaque pli
print("Scores de chaque pli : ", scores)

# Imprimer le score moyen
print("Score moyen : ", scores.mean())


Scores de chaque pli :  [0.88342503 0.97169067 0.98158694 0.97228875 0.95713499]
Score moyen :  0.9532252767311278


Proposition de visualisation

In [13]:
# On souhaite prédire les resultat de l'elction 
# On est au second tour donc on a que 2 candidats MACRON et LE PEN Donc Deux classes
# On va donc utiliser un model de classification
# On va utiliser la regression logistique
loyers_appart = pd.read_csv('https://www.data.gouv.fr/fr/datasets/r/8fac6fb7-cd07-4747-8e0b-b101c476f0da', encoding='iso-8859-1', sep=';')
loyers_appart.head(10)
loyers_maison = pd.read_csv('https://www.data.gouv.fr/fr/datasets/r/cf39b7e5-12fb-48e9-9ee7-b7e5496dcb75', encoding='iso-8859-1', sep=';')

# Convertie les colonnes 'loypredm2' en float
loyers_appart['loypredm2'] = pd.to_numeric(loyers_appart['loypredm2'].str.replace(',', '.'))
loyers_appart['loypredm2'] = pd.to_numeric(loyers_appart['loypredm2'])
loyers_maison['loypredm2'] = pd.to_numeric(loyers_maison['loypredm2'].str.replace(',', '.'))
loyers_maison['loypredm2'] = pd.to_numeric(loyers_maison['loypredm2'])

# Fusionner les deux DataFrames d'origine en un seul DataFrame
loyers = pd.merge(loyers_appart, loyers_maison, on='LIBGEO')

# Calculer la moyenne des loyers prédits pour les appartements et les maisons ligne par ligne
# loyers.head()
loyers['loypredm2_moyen'] = (loyers['loypredm2_x'] + loyers['loypredm2_y']) / 2

# Créer un nouveau DataFrame avec les colonnes 'LIBGEO' et 'loypredm2_moyen'
loyers_moyen = loyers[['LIBGEO', 'loypredm2_moyen']]

# Afficher les 10 premières lignes du nouveau DataFrame
loyers_moyen.head(10)





,LIBGEO,loypredm2_moyen
0,L'Abergement-Clémenciat,8.782018
1,L'Abergement-de-Varey,8.038375
2,Ambérieu-en-Bugey,9.242846
3,Ambérieux-en-Dombes,9.205943
4,Ambléon,8.512610
5,Ambronay,8.240217
6,Ambutrix,8.328916
7,Andert-et-Condon,8.270202
8,Anglefort,10.855396
9,Apremont,8.186342


In [22]:
df_merged = pd.merge(df, loyers_moyen, left_on='Libellé de la commune', right_on='LIBGEO', how='left')
df_merged = df_merged.drop(['LIBGEO'], axis=1)
df_merged.head()


,Libellé du département,Libellé de la commune,Etat saisie,Inscrits,Abstentions,Votants,Blancs,Nuls,Exprimés,Nom,Voix,Unnamed: 28,Unnamed: 30,loypredm2_moyen
0,Ain,L'Abergement-Clémenciat,Complet,643,146,497,42,5,450,MACRON,237,LE PEN,213,8.782018
1,Ain,L'Abergement-de-Varey,Complet,213,45,168,23,3,142,MACRON,94,LE PEN,48,8.038375
2,Ain,Ambérieu-en-Bugey,Complet,8763,2448,6315,461,165,5689,MACRON,3080,LE PEN,2609,9.242846
3,Ain,Ambérieux-en-Dombes,Complet,1282,266,1016,47,15,954,MACRON,451,LE PEN,503,9.205943
4,Ain,Ambléon,Complet,103,19,84,12,0,72,MACRON,45,LE PEN,27,8.512610


In [21]:
df_merged_clean = df_merged.dropna(subset=['loypredm2_moyen'])

X = df_merged_clean[['Nom']]  # variables indépendantes
y = df_merged_clean['loypredm2_moyen']  # variable dépendante

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
r2_score = metrics.r2_score(y_test, y_pred)
scores = cross_val_score(model, X, y, cv=5)

nouvelles_donnees = pd.DataFrame({'Inscrits': [1000], 'Abstentions': [200], 'Blancs': [30], 'Nuls': [10]})
prediction = model.predict(nouvelles_donnees)

# Imprimer les scores de chaque pli
print("Scores de chaque pli : ", scores)

# Imprimer le score moyen
print("Score moyen : ", scores.mean())

print("R²", r2_score)

ValueError: could not convert string to float: 'MACRON'